In [ ]:
import pandas as pd
import workdays
import datetime
from datetime import datetime, timedelta, date

from xbbg import blp
import pdblp
con = pdblp.BCon(timeout=10000)
con.start()

# https://data.bloomberglp.com/labs/sites/2/2013/12/blpapi-developers-guide-1.38.pdf
# https://data.bloomberglp.com/professional/sites/10/2017/03/BLPAPI-Core-Developer-Guide.pdf

In [ ]:
def BDP(L,YK):
    T = []
    T1 = []
    for i in range(len(L)):
        T += [L[i] + "A " + YK[i] ]
        T1 += [L[i] + "1 " + YK[i] ]

    BDP = blp.bdp(tickers=T, flds=["name","currency","country_iso","exch_code","undl_spot_ticker"]).loc[T,:]
    tmp = BDP["undl_spot_ticker"] + " Index"
    tmp = blp.bdp(tickers=tmp, flds="country_iso").loc[tmp,:].iloc[:,0]
    BDP["risk"] = list(tmp)

    BDP.insert(0,"ticker", L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=["contract_value","volume_avg_20d","volatility_20d"]).loc[T1,:]
    BDP1.columns = ["contract_value","volume","vola"]
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)

    BDP = BDP.rename(columns={"currency":"crncy","country_iso":"exch","exch_code":"exch_code","undl_spot_ticker":"undl"})
    
    tmp = ["contract_value","volume"]
    for j in tmp:
        BDP[j] = BDP[j].apply("{:,.0f}".format)
        
    for j in ["vola"]:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(con.bdh(T1[i], "px_last", "19991201", "20220101", elms = [("periodicitySelection","MONTHLY")]).reset_index().iloc[0,0]).strftime("%Y%m%d")]
    BDP["initinal_t/d"] = tmp
    BDP = BDP.replace("19991231", "-")
    
    return BDP
    
L= ["ES","NQ","RTY","VG","GX","Z ","PT","XP","TP","HI","XU","IH","TWT"]
# L= ["ES","NQ"]
YK = ["Index"] * len(L)

print("Equity Futures")
BDP = BDP(L,YK)
print(BDP.shape)
BDP

In [ ]:
def BDP(L,YK):
    T = []
    T1 = []
    for i in range(len(L)):
        T += [L[i] + "A " + YK[i] ]
        T1 += [L[i] + "1 " + YK[i] ]

    BDP = blp.bdp(tickers=T, flds=["name","currency","country_iso","exch_code","fut_ctd_isin"]).loc[T,:]


    isin = BDP.loc[T,["fut_ctd_isin"]].dropna(how='all')
    isin = [i for i in isin["fut_ctd_isin"]]

    tmp0 = ["year","dur","yld"]
    tmp1 = ["mty_years","dur_adj_bid","yld_ytm_bid"]


    BDP1 = blp.bdp(tickers = [i+" Corp" for i in isin], flds=tmp1)
    

    for i in isin:
        for j in range(len(tmp0)):
            BDP.loc[BDP["fut_ctd_isin"]==i,tmp0[j]] = BDP1.loc[i+" Corp",tmp1[j]]

    BDP.insert(0,"ticker", L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=["contract_value","volume_avg_20d","volatility_20d"]).loc[T1,:]
    BDP1.columns = ["contract_value","volume","vola"]
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)
    

    BDP = BDP.rename(columns={"currency":"crncy","country_iso":"exch","exch_code":"exch_code","undl_spot_ticker":"undl"})
    
    for j in ["contract_value","volume"]:
        BDP[j] = BDP[j].apply("{:,.0f}".format)
        
    for j in ["year","dur","yld","vola"]:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(con.bdh(T1[i], "px_last", "19991201", "20210101", elms = [("periodicitySelection","MONTHLY")]).reset_index().iloc[0,0]).strftime("%Y%m%d")]
    BDP["initinal_t/d"] = tmp
    BDP = BDP.replace("19991231", "-")
    
    return BDP
    
L = ["TU","FV","TY","US","WN","DU","OE","RX","UB","OAT","IK","G ","CN","XM","JB"]
# L = ["TU","FV"]
YK = ["Comdty"] * len(L)

print("Bond Futures")
BDP = BDP(L,YK)
print(BDP.shape)
BDP

In [ ]:
def BDP(L,YK):
    T = []
    T1 = []
    for i in range(len(L)):
        T += [L[i] + "A " + YK[i] ]
        T1 += [L[i] + "1 " + YK[i] ]

    BDP = blp.bdp(tickers=T, flds=["name","currency","country_iso","exch_code","undl_spot_ticker"]).loc[T,:]

    BDP.insert(0,"ticker", L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=["contract_value","volume_avg_20d","volatility_20d"]).loc[T1,:]
    BDP1.columns = ["contract_value","volume","vola"]
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)

    BDP = BDP.rename(columns={"currency":"crncy","country_iso":"exch","exch_code":"exch_code","undl_spot_ticker":"undl"})
    
    tmp = ["contract_value","volume"]
    for j in tmp:
        BDP[j] = BDP[j].apply("{:,.0f}".format)
        
    for j in ["vola"]:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(con.bdh(T1[i], "px_last", "19991201", "20220101", elms = [("periodicitySelection","MONTHLY")]).reset_index().iloc[0,0]).strftime("%Y%m%d")]
    BDP["initinal_t/d"] = tmp
    BDP = BDP.replace("19991231", "-")
    
    return BDP
    
L= ["CL","CO","GC"]
YK = ["Comdty"] * len(L)

print("Commodity Futures")
BDP = BDP(L,YK)
print(BDP.shape)
BDP

In [ ]:
def BDP(L):
    T = ["CSBS" + i + " TMUQ Curncy" for i in L]  #Mitsubishi TTM
    tmp = ["px_last","volatility_60d","rsi_30d"]
    BDP = blp.bdp(tickers=T, flds=tmp).loc[T,:]
    BDP = BDP.loc[T,:]

    for i in tmp:
        BDP[i.lower()] = round(BDP[i.lower()].astype(float),2)
    BDP.insert(0, "currency", L )
    BDP.insert(1, "ticker", T)
    BDP = BDP.reset_index(drop=True)
    BDP = BDP.rename(columns={"volatility_60d":"vola"})

    return BDP

L = ["USD","EUR","GBP","CAD","AUD","NZD","CHF","NOK","SEK","HKD","CNY"]

print("FX")
BDP = BDP(L)
print(BDP.shape)
BDP

In [ ]:
def BDP(L):
    T = [i + " Index" for i in L]   
    BDP = blp.bdp(tickers=T, flds=["Security_name","long_comp_name","Currency","volatility_260d"]).loc[T,:]
    BDP.insert(0, "ticker", T)
    BDP = BDP.reset_index(drop=True)
    BDP = BDP.rename(columns={"volatility_260d":"vola"})
    BDP["vola"] = round(BDP["vola"].astype(float),2)

    for i in range(len(BDP)):
        tmp = con.bdh(BDP.loc[i,"ticker"], ["px_last","tot_return_index_gross_dvds"], "20000101", "20230301")
        
        BDP.loc[i,"PR"] = round(tmp.iloc[len(tmp)-1,0] / tmp.iloc[0,0] - 1,2) 
        BDP.loc[i,"TR"] = round(tmp.iloc[len(tmp)-1,1] / tmp.iloc[0,1] - 1,2) 

        if tmp.iloc[len(tmp)-1,1] - tmp.iloc[len(tmp)-1,0] == 0:
            BDP.loc[i,"PR=TR"] = "1"
        else:
            BDP.loc[i,"PR=TR"] = ""

        BDP.loc[i,"Initial_TD"] = tmp.index[0].strftime("%Y%m%d")
    BDP = BDP.replace("20000104", "-")
    return BDP
# https://www.jpmorgan.com/insights/research/index-research
# https://www.jpmorgan.com/content/dam/jpm/cib/complex/content/markets/index-research/Global-Index-Research-Product-Guide-2022.pdf#page=117

In [ ]:
print("Japan Stocks")
L = ["TPX","TPXDDVD"]
BDP(L)

In [ ]:
print("Developed Country Stocks")
L = ["MBKO","M0KO","MXKO","M2KO","MAKO","MSHJKOK","GJ124280","HJ124336"]
BDP(L)

In [ ]:
print("Emerging Markets Stocks")
L = ["GDUEEGF","NDUEEGF","HJ137561"]
BDP(L)

In [ ]:
print("Developed Country REIT")
L = ["SREITWJJ","SREITJWJ"]
BDP(L)

In [ ]:
print("Japan Bonds")
L = ["BPITTO01"]
BDP(L)

In [ ]:
print("Developed Country Bond")
L = ["SBWGNJYC","SBWGNJYU"]
BDP(L)

In [14]:
print("Emerging Market Bond / USD Bond")
L = ["JPGCUJCP","JPGCHJCP","JPEIDIVR","JPGCCOMP"]
# L = ["JPGCUJCP","JPGCHJCP","JPEIDIVR","JPEIPLUS","JPEMCOMP","GBIE1545"]
tmp = BDP(L)
tmp

Emerging Market Bond / USD Bond


,ticker,security_name,long_comp_name,currency,volatility_260d,PR,TR,PR=TR,Initial_TD
0,JPGCUJCP Index,unhedged Return JPY,J.P. Morgan EMBI Global Diversified unhedged R...,JPY,12.33011,1.10,1.10,1,20121001
1,JPGCHJCP Index,Hedged JPY,J.P. Morgan EMBI Global Diversified Hedged JPY,JPY,8.401531,1.89,1.89,1,20000103
2,JPEIDIVR Index,JPMorgan Monthly EMBIs,J.P. Morgan EMBI Global Diversified Composite,USD,8.378298,3.68,3.68,1,20000103
3,JPGCCOMP Index,Composite,J.P. Morgan EMBI Global Diversified Composite,USD,8.348022,3.68,3.68,1,20000103


In [15]:
print("Emerging Market Bond / Local Currency Bond")
L = ["JGENDVUJ","JGENDVHJ","JGENGUJG","JGENGHJG","JGENVUJG","JGENVHJG"]
tmp = BDP(L)
tmp["Remarks"] = ["GBI EM Div"]*2+["GBI EM Global"]*2+["GBI EM Global Div"]*2
tmp

Emerging Market Bond / Local Currency Bond


,ticker,security_name,long_comp_name,currency,volatility_260d,PR,TR,PR=TR,Initial_TD,Remarks
0,JGENDVUJ Index,Divers,J.P. Morgan GBI-EM Diversified JPY Unhedged,JPY,12.088974,1.73,1.73,1,20021231,GBI EM Div
1,JGENDVHJ Index,Divers,J.P. Morgan GBI-EM Diversified JPY Hedged,JPY,3.710679,0.27,0.27,1,20030101,GBI EM Div
2,JGENGUJG Index,JPY,J.P. Morgan GBI-EM Global Composite Unhedged JPY,JPY,11.754041,2.57,2.57,1,20011231,GBI EM Global
3,JGENGHJG Index,JPY,J.P. Morgan GBI-EM Global Composite Hedged JPY,JPY,1.62721,0.53,0.53,1,20011231,GBI EM Global
4,JGENVUJG Index,JPY,J.P. Morgan GBI-EM Global Diversified Composit...,JPY,11.364509,2.04,2.04,1,20021231,GBI EM Global Div
5,JGENVHJG Index,JPY,J.P. Morgan GBI-EM Global Diversified Composit...,JPY,2.999868,0.39,0.39,1,20021231,GBI EM Global Div
